In [52]:
# newsAPI     =    0b1ea46e336d40e1abc48f51e01c253a

In [1]:
import pandas as pd
import numpy as np
from transformers import pipeline
from tqdm import tqdm
from collections import defaultdict
from datetime import timedelta


## load the data

In [26]:
df = pd.read_csv('testData.csv')

# convert the publish_date column to datetime64
df['publish_date'] = pd.to_datetime(df['publish_date'])

# convert remaining cols to string
df['headline_text'] = df['headline_text'].astype(str)
df['description'] = df['description'].astype(str)


headlines_df = df.copy()
headlines_df['headline_sentiment'] = 0
headlines_df['description_sentiment'] = 0
headlines_df['emotion'] = 0
# headlines_df['pos_sentiment'] = 0
# headlines_df['neg_sentiment'] = 0

print(headlines_df.dtypes)
headlines_df.head()

total_headlines = len(headlines_df)

publish_date             datetime64[ns, UTC]
headline_text                         object
description                           object
headline_sentiment                     int64
description_sentiment                  int64
emotion                                int64
dtype: object


In [23]:
valid_emotions = ['joy', 'others', 'surprise',
                  'sadness', 'fear', 'anger', 'disgust']


# emotions =  {"joy": 0, "others": 0, "surprise": 0, 
#              "sadness": 0, "fear": 0, "anger": 0, "disgust": 0,
#              "others": 0}

emotions = {}
# emotions = defaultdict(int)

# print(emotions.keys())
# print(emotions)

## load the models

In [12]:
# sentiments = defaultdict(lambda: defaultdict(int))
sentiment_model = pipeline(
    model="siebert/sentiment-roberta-large-english")  # Use device 0 for GPU

emotion_model = pipeline(
    model="finiteautomata/bertweet-base-emotion-analysis")

# keyword_ext_model = pipeline(
#     model="yanekyuk/bert-keyword-extractor")

# keyword_senti_model = pipeline(
#     model="cardiffnlp/twitter-roberta-base-sentiment-latest")


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at siebert/sentiment-roberta-large-english.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassific

In [27]:
## analyse headlines ##
with tqdm(total=total_headlines, desc="Analysing Sentiments", unit="headline", dynamic_ncols=True) as pbar:
    for idx in range(total_headlines):
        row = headlines_df.iloc[idx]
        headline = row['headline_text']
        description = row['description']


        result = sentiment_model(headline)
        label = result[0]['label']

        if label == 'POSITIVE':
            headlines_df.at[idx, 'headline_sentiment'] = 1
        elif label == 'NEGATIVE':
            headlines_df.at[idx, 'headline_sentiment'] = -1

        ## analyse description ##               
        result = sentiment_model(description)
        label = result[0]['label']

        if label == 'POSITIVE':
            headlines_df.at[idx, 'description_sentiment'] = 1
        elif label == 'NEGATIVE':
            headlines_df.at[idx, 'description_sentiment'] = -1
        

        ## analyse emotions ##
        results = emotion_model(headline)
        # print(results)

        for result in results:
            label = result['label']
            if label in valid_emotions:

                if label not in emotions:
                    emotions[label] = 0

                emotions[label] += 1

                headlines_df.at[idx, "emotion"] = label

        pbar.update(1)

Analysing Sentiments: 100%|██████████| 5/5 [00:07<00:00,  1.44s/headline]


In [28]:
headlines_df.head()

# print(emotions)

,publish_date,headline_text,description,headline_sentiment,description_sentiment,emotion
0,2024-01-28 04:50:27+00:00,Elon Musk joins Trump Republicans to slam rumo...,Elon Musk joined Donald Trump and Republican c...,-1,-1,others
1,2024-01-28 00:10:36+00:00,Tesla battery explodes in Cary home after bein...,Tesla battery explodes in Cary home after bein...,-1,-1,others
2,2024-01-27 21:01:05+00:00,Here's why Biden's multi-billion-dollar EV cha...,By Will Kessler Daily Caller News Foundation T...,-1,-1,others
3,2024-01-27 17:30:00+00:00,Who wants to be a trillionaire': The game show...,A trillion dollars can purchase shares of all...,1,1,others
4,2024-01-27 16:00:00+00:00,ARGHHH FUCK THIS,test,-1,1,anger


In [22]:
print(emotions)

{'others': 8, 'anger': 2}
